In [1]:
from google.colab import drive
drive.mount('/content/drive/')

!cp "/content/drive/My Drive/Dissertation/envs/point_push.py" .

Mounted at /content/drive/


In [2]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_push_hdqn/{name}" 

    torch.save({
      'meta_controller': model.meta_controller.state_dict(),
      'controller': {
          'critic': model.controller.critic.state_dict(),
          'actor': model.controller.actor.state_dict(),
      }
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/point_push_hdqn/{name}" 
    checkpoint = torch.load(path)

    model.meta_controller.load_state_dict(checkpoint['meta_controller'])
    model.meta_controller_target = copy.deepcopy(model.meta_controller)

    model.controller.critic.load_state_dict(checkpoint['controller']['critic'])
    model.controller.critic_target = copy.deepcopy(model.controller.critic)
    model.controller.actor.load_state_dict(checkpoint['controller']['actor'])
    model.controller.actor_target = copy.deepcopy(model.controller.actor)

    # model.eval() for evaluation instead
    model.eval()
    model.meta_controller.eval()
    model.controller.eval()

In [3]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [4]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [5]:
from point_push import PointPushEnv 
env = NormalizedEnv(PointPushEnv(4))

***

In [6]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [7]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [8]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
  
# (state, action) -> (next_state, reward, done)
transition_meta = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done', 'state_seq', 'action_seq'))

# replay memory D with capacity N
class ReplayMemoryMeta(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition_meta(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [9]:
DEPTH = 128

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, DEPTH)
        self.fc2 = nn.Linear(DEPTH, DEPTH)
        self.head = nn.Linear(DEPTH, nb_actions)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, DEPTH)
        self.l2 = nn.Linear(DEPTH, DEPTH)
        self.l3 = nn.Linear(DEPTH, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, DEPTH)
        self.l5 = nn.Linear(DEPTH, DEPTH)
        self.l6 = nn.Linear(DEPTH, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [10]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions, is_meta=False):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        self.is_meta = is_meta

        #Create replay buffer
        self.memory = ReplayMemory(100000) if not self.is_meta else ReplayMemoryMeta(100000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 10000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.is_training = True

    def update_policy(self, off_policy_correction=None):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)

        if not self.is_meta:
            batch = transition(*zip(*transitions))
            action_batch = torch.cat(batch.action)
        else:
            batch = transition_meta(*zip(*transitions))

            action_batch = torch.cat(batch.action)
            state_seq_batch = torch.stack(batch.state_seq)
            action_seq_batch = torch.stack(batch.action_seq)

            action_batch = off_policy_correction(action_batch.cpu().numpy(), state_seq_batch.cpu().numpy(), action_seq_batch.cpu().numpy())
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
      
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # print losses
            #if self.total_it % (50 * 50 if self.is_meta else 500 * 50) == 0:
            #    print(f"{self.is_meta} controller;\n\tcritic loss: {critic_loss.item()}\n\tactor loss: {actor_loss.item()}")

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, 2 * self.tau / 5)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, s_t, a_t, s_t1, r_t, done):
        self.memory.store(s_t, a_t, s_t1, r_t, done)

    def random_action(self):
        return torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)

    def select_action(self, s_t, warmup, decay_epsilon):
        if warmup:
            return self.random_action()

        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            #action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * np.random.uniform(-1.,1.,1)).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            return action

class DQN(nn.Module):
    def __init__(self, inputs, outputs, mem_len = 100000):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(inputs, DEPTH)
        self.fc2 = nn.Linear(DEPTH, DEPTH)
        self.head = nn.Linear(DEPTH, outputs)
        
        self.memory = ReplayMemory(mem_len)

        self.n_actions = outputs
        self.steps_done = 0
        
        self.EPS_START = 1.0
        self.EPS_END = 0.0
        self.EPS_DECAY = 10000 # in number of steps
        self.TAU = 0.001

        self.eps_printed = False

        self.policy_update = 2
        self.tot_updates = 0

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.head(x)
    
    def act(self, state, warmup, is_training):
        if warmup: 
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

        if is_training:
            eps_threshold = self.EPS_END + (self.EPS_START - self.EPS_END) * (1. - min(1., self.steps_done / self.EPS_DECAY))
            self.steps_done += 1

            if eps_threshold <= 0.2 and not self.eps_printed:
                self.eps_printed = True
                print("EPS_THRESHOLD below 0.2")

            # With probability eps select a random action
            if random.random() < eps_threshold:
                return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

        # otherwise select action = maxa Q∗(φ(st), a; θ)
        with torch.no_grad():
            return self(state).max(1)[1].view(1, 1)
    
    def experience_replay(self, optimizer, target):
        if len(self.memory) < BATCH_SIZE:
            return

        self.tot_updates += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)
        
        current_Q_values = self(state_batch).gather(1, action_batch)
        # Compute next Q value based on which goal gives max Q values
        # Detach variable from the current graph since we don't want gradients for next Q to propagated
        next_max_q = target(next_state_batch).detach().max(1)[0]
        next_Q_values = not_done_mask * next_max_q
        # Compute the target of the current Q values
        target_Q_values = reward_batch + (GAMMA * next_Q_values)
        # Compute Bellman error (using Huber loss)
        loss = F.smooth_l1_loss(current_Q_values, target_Q_values.unsqueeze(1))
        loss_val = loss.item()

        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        for param in self.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()

        if self.tot_updates % self.policy_update == 0:
            soft_update(target, self, self.TAU)

        return loss_val

In [11]:
class HIRO(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(HIRO, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        self.goal_dim = [0, 1]
        self.goal_dimen = 2
      
        learning_rate = 2.5e-4
        self.meta_controller = DQN(nb_states, 11 * 11).to(device)
        self.meta_controller_optimizer = optim.RMSprop(self.meta_controller.parameters(), lr=learning_rate)
        self.meta_controller_target = DQN(nb_states, 11 * 11, mem_len = 0).to(device)
        self.meta_controller_target.eval()

        self.max_goal_dist = torch.from_numpy(np.array([2.5, 2.5])).to(device)
        self.goal_offset = torch.from_numpy(np.array([0., 1.])).to(device)

        self.controller = TD3(nb_states + len(self.goal_dim), nb_actions).to(device)
        #self.controller.depsilon = 1.0 / 10000

    def teach_controller(self):
        self.controller.update_policy()
    def teach_meta_controller(self):
        self.meta_controller.experience_replay(self.meta_controller_optimizer, self.meta_controller_target)

    def h(self, state, goal, next_state):
        #return goal
        return state[:,self.goal_dim] + goal - next_state[:,self.goal_dim]
    #def intrinsic_reward(self, action, goal):
    #    return torch.tensor(1.0 if self.goal_reached(action, goal) else 0.0, device=device) 
    #def goal_reached(self, action, goal, threshold = 0.1):
    #    return torch.abs(action - goal) <= threshold
    def intrinsic_reward(self, reward, state, goal, next_state):
        #return torch.tensor(2 * reward if self.goal_reached(state, goal, next_state) else reward / 10, device=device) #reward / 2
        # just L2 norm
        return -torch.pow(sum(torch.pow(state.squeeze(0)[self.goal_dim] + goal.squeeze(0) - next_state.squeeze(0)[self.goal_dim], 2)), 0.5)
    def goal_reached(self, state, goal, next_state, threshold = 0.1):
        return torch.pow(sum(torch.pow(state.squeeze(0)[self.goal_dim] + goal.squeeze(0) - next_state.squeeze(0)[self.goal_dim], 2)), 0.5) <= threshold
        #return torch.pow(sum(goal.squeeze(0), 2), 0.5) <= threshold

    def observe_controller(self, s_t, a_t, s_t1, r_t, done):
        self.controller.memory.store(s_t, a_t, s_t1, r_t, done)
    def observe_meta_controller(self, s_t, a_t, s_t1, r_t, done, state_seq, action_seq):
        self.meta_controller.memory.store(s_t, a_t, s_t1, r_t, done)

    def action_to_2D(self, a):
        x = (a % 11)
        y = (a // 11)
        return -1.0 + 0.2 * torch.cat([x, y], axis=1).float()

    def convert_goal(self, a):
        return self.action_to_2D(a) * self.max_goal_dist + self.goal_offset

    def select_goal(self, s_t, warmup, is_training):
        return self.meta_controller.act(s_t, warmup, is_training)
      
    def select_action(self, s_t, g_t, warmup, decay_epsilon):
        sg_t = torch.cat([s_t, g_t], 1).float()
        return self.controller.select_action(sg_t, warmup, decay_epsilon)

In [ ]:
import time
SAVE_OFFSET = 4
def train_model():
    global SAVE_OFFSET
    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = HIRO(n_observations, n_actions).to(device)
    
    max_episode_length = 500
    
    agent.is_training = True
    episode_reward = 0.
    observation = None
    
    warmup = 100
    num_episodes = 12000 # M
    episode_durations = []
    goal_durations = []

    steps = 0
    c = 10

    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        
        overall_reward = 0
        overall_intrinsic = 0
        episode_steps = 0
        done = False
        goals_done = 0

        while not done:
            goal_raw = agent.select_goal(state, i_episode <= warmup, True)
            goal = agent.convert_goal(goal_raw)
            #goal_durations.append((steps, goal[:,0]))

            state_seq, action_seq = None, None
            first_goal = goal
            goal_done = False
            total_extrinsic = 0

            while not done and not goal_done:
                joint_goal_state = torch.cat([state, goal], axis=1).float()

                # agent pick action ...
                action = agent.select_action(state, goal, i_episode <= warmup, True)
                
                # env response with next_observation, reward, terminate_info
                observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                steps += 1
                next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                next_goal = agent.h(state, goal, next_state)
                joint_next_state = torch.cat([next_state, next_goal], axis=1).float()
                
                if max_episode_length and episode_steps >= max_episode_length -1:
                    done = True
                    
                extrinsic_reward = torch.tensor([reward], device=device)
                intrinsic_reward = agent.intrinsic_reward(reward, state, goal, next_state).unsqueeze(0)
                #intrinsic_reward = agent.intrinsic_reward(action, goal).unsqueeze(0)

                overall_reward += reward
                total_extrinsic += reward
                overall_intrinsic += intrinsic_reward

                goal_reached = agent.goal_reached(state, goal, next_state)
                #goal_done = agent.goal_reached(action, goal)

                # agent observe and update policy
                agent.observe_controller(joint_goal_state, action, joint_next_state, intrinsic_reward, done) #goal_done.item())

                if state_seq is None:
                    state_seq = state
                else:
                    state_seq = torch.cat([state_seq, state])
                if action_seq is None:
                    action_seq = action
                else:
                    action_seq = torch.cat([action_seq, action])

                episode_steps += 1

                if goal_reached:
                    goals_done += 1
                
                if (episode_steps % c) == 0:
                    agent.observe_meta_controller(state_seq[0].unsqueeze(0), goal_raw, next_state, torch.tensor([total_extrinsic], device=device), done,\
                                                  state_seq, action_seq)
                    goal_done = True

                    if i_episode > warmup:
                        agent.teach_meta_controller()

                state = next_state
                goal = next_goal
                
                if i_episode > warmup:
                    agent.teach_controller()

        goal_durations.append((i_episode, overall_intrinsic / episode_steps))
        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations, goal_durations)

        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 100 == 0:
                print(f"{i_episode}: {np.mean(dur[-100:])}")
            if i_episode >= 300 and i_episode % 100 == 0 and np.mean(dur[-100:]) <= -47.0:
                print(f"Unlucky after {i_episode} eps! Terminating...")
                return None
            if np.mean(dur[-100:]) >= 90:
                print(f"Solved after {i_episode} episodes!")
                save_model(agent, f"hiro_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return agent

    return None # did not train

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def eval_model(agent, episode_durations, goal_attack, action_attack, same_noise):
    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.meta_controller.is_training = False
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for l2norm in np.arange(0.0,0.51,0.05):
        
        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)

            if goal_attack:
                g_state = g_state + state_range * noise
                g_state = torch.max(torch.min(g_state, state_max), state_min).float()
            if action_attack:
                if same_noise:
                    state = state + state_range * noise
                else:
                    state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                state = torch.max(torch.min(state, state_max), state_min).float()

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal_raw = agent.select_goal(state, False, False)
                goal = agent.convert_goal(goal_raw)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                    
                    next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                    if goal_attack:
                        g_next_state = g_next_state + state_range * noise
                        g_next_state = torch.max(torch.min(g_next_state, state_max), state_min).float()
                    if action_attack:
                        if same_noise:
                            next_state = next_state + state_range * noise
                        else:
                            next_state = next_state + state_range * torch.FloatTensor(next_state.shape).uniform_(-l2norm, l2norm).to(device)
                        next_state = torch.max(torch.min(next_state, state_max), state_min).float()

                    next_goal = agent.h(g_state, goal, g_next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    g_state = g_next_state
                    goal = next_goal

        episode_durations[np.round(l2norm, 2)].append(overall_reward / num_episodes)

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data + eps * sign_data_grad * state_range

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_goal(g_state, agent, eps, target, targeted):
    g_state = g_state.clone().detach().requires_grad_(True)

    # initial forward pass
    goal = agent.meta_controller(g_state)

    if targeted:
        loss = F.mse_loss(goal, target)
    else:
        loss = F.mse_loss(goal, -goal) # doing -goal inverts the argmax 

    agent.meta_controller.zero_grad()

    # calc loss
    loss.backward()
    data_grad = g_state.grad.data

    # perturb state
    g_state_p = fgsm_attack(g_state, eps, data_grad).float()
    return g_state_p

def fgsm_action(state, goal, agent, eps, target, targeted):
    #state = torch.tensor(state, requires_grad=True)
    state = state.clone().detach().requires_grad_(True)
    goal = goal.clone().detach()

    sg_t = torch.cat([state, goal], 1).float()

    if targeted:
        # initial forward pass
        action = agent.controller.actor(sg_t)
        action = torch.clamp(action, -1., 1.)

        loss = F.mse_loss(action, target)
    else:
        loss = agent.controller.critic.Q1(sg_t, agent.controller.actor(sg_t)).mean()

    agent.controller.actor.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad).float()
    return state_p

def apply_fgsm(agent, episode_durations, goal_attack, targeted):
    TARGET_GOAL = torch.tensor([[0.0] * (11 * 11)], device=device, dtype=torch.float)
    TARGET_ACTION = torch.tensor([[0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.meta_controller.is_training = False
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for eps in np.arange(0.0, 0.201, 0.02):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            og_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            if goal_attack: # target meta controller
                state = fgsm_goal(og_state, agent, eps, TARGET_GOAL, targeted)
            else: # target controller
                goal_raw = agent.select_goal(og_state, False, False)
                goal = agent.convert_goal(goal_raw)
                state = fgsm_action(og_state, goal, agent, eps, TARGET_ACTION, targeted)

            episode_steps = 0
            done = False
            while not done:
                goal_raw = agent.select_goal(state, False, False)
                goal = agent.convert_goal(goal_raw)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())

                    next_og_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    if goal_attack: # target meta controller
                        next_state = fgsm_goal(next_og_state, agent, eps, TARGET_GOAL, targeted)
                    else: # target controller
                        goal_temp = agent.h(state, goal, next_og_state)
                        next_state = fgsm_action(next_og_state, goal_temp, agent, eps, TARGET_ACTION, targeted)

                    next_goal = agent.h(state, goal, next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(state, goal, next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    goal = next_goal

        episode_durations[eps].append(overall_reward / num_episodes)

In [ ]:
noise_hrl = {'both': {}, 'action_only': {}, 'goal_only': {}, 'both_same': {}}
for l2norm in np.arange(0,0.51,0.05):
    for i in [noise_hrl['both'], noise_hrl['action_only'], noise_hrl['goal_only'], noise_hrl['both_same']]:
        i[np.round(l2norm, 2)] = []

targeted = {'both': {}, 'goal_only': {}, 'action_only': {}}
untargeted = {'both': {}, 'goal_only': {}, 'action_only': {}}
for eps in np.arange(0.0, 0.201, 0.02):
    for x in ['both', 'goal_only', 'action_only']:
        targeted[x][eps] = []
        untargeted[x][eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 4
while i < 8:
    if i == 0:
        agent = HIRO(n_observations, n_actions).to(device)
        load_model(agent, f"hiro_{i}")
    else:
        agent = train_model()

    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_model(agent, noise_hrl['both_same'], True, True, True)
        eval_model(agent, noise_hrl['both'], True, True, False)
        eval_model(agent, noise_hrl['action_only'], False, True, False)
        eval_model(agent, noise_hrl['goal_only'], True, False, False)
        print(f"{i} noise_hrl: {noise_hrl}")
        i += 1

print("----")
print(f"noise_hrl: {noise_hrl}")

#i = 0
#while i < 13:
#    #agent = train_model()
#    agent = HIRO(n_observations, n_actions).to(device)
#    load_model(agent, f"hiro_push_{i}")

#    if agent is not None:   
#      apply_fgsm(agent, targeted['both'], True, True, False)
#      apply_fgsm(agent, targeted['goal_only'], True, False, False)
#      apply_fgsm(agent, targeted['action_only'], False, True, False)   
#      print(f"{i} fgsm (ut): {targeted}")
#      i += 1

#print("----")
#print(f"fgsm (ut): {targeted}")

100: -46.44100000000042
200: -26.193000000000378
EPS_THRESHOLD below 0.2
300: -21.81700000000036
400: -37.493000000000414
500: -34.3750000000004
600: -35.12900000000041
700: -25.10600000000038
800: -27.10400000000038
900: -28.93700000000038
1000: -34.7490000000004
1100: -28.71900000000039
1200: -34.4300000000004
1300: -20.742000000000356
1400: -25.256000000000377
1500: -37.79300000000041
1600: -27.87500000000039
1700: -14.842000000000347
1800: -25.451000000000377
1900: -18.357000000000344
2000: -27.702000000000382
2100: -22.606000000000357
2200: -18.931000000000353
2300: 3.1949999999997085
2400: -14.388000000000345
2500: -6.6610000000003176
2600: 3.875999999999715
2700: -8.789000000000321
2800: 5.215999999999724
2900: -2.694000000000301
3000: -4.196000000000307
3100: -6.3280000000003085
3200: -18.801000000000343
3300: -0.895000000000296
3400: 2.9479999999997206
3500: 6.047999999999729
3600: -8.469000000000314
3700: 19.055999999999763
3800: -13.672000000000333
3900: -6.35200000000032
40

In [ ]:
targeted = {'goal': {}, 'action': {}}
untargeted = {'goal': {}, 'action': {}}
for eps in np.arange(0.0, 0.201, 0.02):
    for x in ['goal', 'action']:
        targeted[x][eps] = []
        untargeted[x][eps] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

for i in [0, 1, 4, 5, 6]:
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_{i}")

    if agent is not None:
        apply_fgsm(agent, untargeted['action'], False, False)   
        apply_fgsm(agent, untargeted['goal'], True, False)  
        print(f"{i} fgsm (ut): {untargeted}")

        apply_fgsm(agent, targeted['goal'], True, True)
        apply_fgsm(agent, targeted['action'], False, True)   
        print(f"{i} fgsm (t): {targeted}")

print("----")
print(f"fgsm (ut): {untargeted}")
print(f"fgsm (t): {targeted}")

0 fgsm (ut): {'goal': {0.0: [98.03099999999594], 0.02: [67.78499999998444], 0.04: [15.729000000008236], 0.06: [7.596999999998364], 0.08: [23.9560000000084], 0.1: [5.371999999994931], 0.12: [0.7759999999957947], 0.14: [-12.347000000003339], 0.16: [-43.16099999998398], 0.18: [-50.00000000000659], 0.2: [-48.61100000000062]}, 'action': {0.0: [97.63399999999696], 0.02: [45.09299999998916], 0.04: [-12.89999999999932], 0.06: [-27.618999999973934], 0.08: [-33.438999999973895], 0.1: [-39.112999999971244], 0.12: [-50.00000000000659], 0.14: [-50.00000000000659], 0.16: [-48.82600000000141], 0.18: [-50.00000000000659], 0.2: [-48.758000000001715]}}
0 fgsm (t): {'goal': {0.0: [97.87899999999692], 0.02: [78.79099999998121], 0.04: [16.35000000000866], 0.06: [22.161000000009256], 0.08: [-3.3770000000015923], 0.1: [5.5679999999952665], 0.12: [-5.340000000004885], 0.14: [-20.358999999976014], 0.16: [-43.30799999997782], 0.18: [-48.61800000000065], 0.2: [-46.027999999989405]}, 'action': {0.0: [97.705999999

In [ ]:
Solved after 9695 episodes!
Solved after 9134 episodes!
1 noise_hrl: {'both': {0.0: [97.46999999999527, 97.69699999999574], 0.05: [86.23699999997838, 56.576999999990136], 0.1: [34.21900000001736, 33.15700000001119], 0.15: [-9.768000000006756, 6.446999999996693], 0.2: [-29.58099999997852, -15.552999999994652], 0.25: [-35.631999999971725, -19.1929999999875], 0.3: [-42.43899999997449, -21.05499999999158], 0.35: [-44.99099999998676, -21.065999999985042], 0.4: [-38.34199999997063, -32.733999999971054], 0.45: [-36.190999999971616, -21.699999999978385], 0.5: [-31.057999999972775, -28.448999999974166]}, 'action_only': {0.0: [98.20199999999659, 97.79299999999597], 0.05: [73.04099999998135, 71.98599999999087], 0.1: [38.689000000011454, 32.16200000001837], 0.15: [-0.5350000000020939, -2.0710000000045357], 0.2: [-28.905999999974846, -14.108999999994742], 0.25: [-39.94599999997083, -24.54999999997766], 0.3: [-36.11799999997128, -34.81699999997497], 0.35: [-41.80899999997474, -34.2489999999706], 0.4: [-43.04899999997761, -34.53999999997176], 0.45: [-29.11599999997332, -41.22199999996942], 0.5: [-43.45199999997935, -26.097999999978345]}, 'goal_only': {0.0: [97.53699999999459, 97.6939999999958], 0.05: [98.25999999999671, 98.05999999999632], 0.1: [98.36199999999695, 98.1469999999968], 0.15: [96.85099999999518, 98.12799999999648], 0.2: [95.13499999999601, 91.58299999999376], 0.25: [95.36699999999344, 86.6709999999874], 0.3: [86.22899999998955, 78.81199999998665], 0.35: [85.8729999999934, 74.61299999999528], 0.4: [81.7329999999885, 69.98499999998192], 0.45: [81.7779999999887, 58.76000000000011], 0.5: [70.47899999999181, 71.94199999998554]}, 'both_same': {0.0: [97.86399999999561, 97.66499999999573], 0.05: [90.178999999989, 60.0349999999687], 0.1: [49.79799999999771, 37.18500000001147], 0.15: [10.969999999997752, 14.20000000000422], 0.2: [-23.030999999982704, -4.737000000005985], 0.25: [-40.16399999997147, -19.314999999983023], 0.3: [-40.1219999999681, -36.48299999997436], 0.35: [-40.83799999997313, -31.239999999976366], 0.4: [-33.625999999974, -31.201999999970923], 0.45: [-36.96799999997457, -26.946999999970966], 0.5: [-36.085999999975414, -28.83799999997012]}}
Solved after 7390 episodes!
Solved after 7360 episodes!
3 noise_hrl: {'both': {0.0: [42.21600000002324, 89.2129999999878], 0.05: [44.11300000000775, 59.10999999999389], 0.1: [12.290000000006183, 21.485000000007112], 0.15: [-11.47399999999868, 12.17300000000335], 0.2: [-21.357999999987527, -14.231999999991933], 0.25: [-29.883999999980897, -34.39899999997501], 0.3: [-26.52299999997229, -41.429999999970185], 0.35: [-33.2929999999718, -40.732999999968676], 0.4: [-35.60199999997248, -35.090999999972176], 0.45: [-32.73199999997401, -32.08699999997541], 0.5: [-26.674999999980535, -27.920999999976015]}, 'action_only': {0.0: [44.753000000001094, 96.66499999999478], 0.05: [49.01400000000039, 36.93600000001564], 0.1: [18.860000000005904, 25.844000000010254], 0.15: [1.4450000000000633, 14.938000000018473], 0.2: [-13.440000000000044, -16.726999999979597], 0.25: [-20.431999999992648, -15.856999999995166], 0.3: [-32.033999999973396, -35.18899999997122], 0.35: [-30.409999999977835, -36.757999999972974], 0.4: [-25.24799999998376, -37.96999999997074], 0.45: [-32.09999999997583, -26.849999999977143], 0.5: [-26.56199999997526, -30.726999999975483]}, 'goal_only': {0.0: [46.24700000000135, 95.1619999999954], 0.05: [75.61299999997323, 92.50799999999013], 0.1: [62.96099999998537, 92.63899999999624], 0.15: [45.817000000001755, 91.18499999999054], 0.2: [49.66199999999173, 92.50699999999227], 0.25: [25.193000000019634, 91.01199999998856], 0.3: [42.9039999999893, 80.4069999999889], 0.35: [29.879000000019303, 80.53099999998716], 0.4: [21.221000000011667, 78.42299999997904], 0.45: [44.065000000004495, 69.70099999998529], 0.5: [28.343000000018396, 72.68399999998843]}, 'both_same': {0.0: [35.32200000002367, 95.13299999999538], 0.05: [48.64299999998538, 35.830000000012376], 0.1: [14.840000000015566, 40.69000000001639], 0.15: [4.180999999993211, 15.139000000013262], 0.2: [-3.9700000000009963, -5.374000000005266], 0.25: [-6.674000000001981, -35.676999999971684], 0.3: [-27.005999999975828, -37.28099999997355], 0.35: [-31.894999999972114, -37.451999999972855], 0.4: [-27.572999999979796, -38.20699999996802], 0.45: [-35.51899999996971, -33.39299999997358], 0.5: [-33.37699999997492, -29.324999999976143]}}
Solved after 7772 episodes!
Solved after 8822 episodes!
Solved after 5457 episodes!
Solved after 8211 episodes!
7 noise_hrl: {'both': {0.0: [96.1429999999931, 98.13499999999645, 93.7829999999928, 43.371000000012764], 0.05: [41.2690000000128, 14.969000000011869, 16.188000000003534, 97.21999999999473], 0.1: [32.656000000012774, -13.789999999993347, 32.09900000001799, 73.49499999997947], 0.15: [9.064999999993674, -19.634999999983094, 7.805999999995798, 32.85100000001008], 0.2: [1.0399999999992025, -11.475999999998827, 14.737000000010987, 11.570000000009548], 0.25: [10.563000000001237, -13.063999999991879, -16.36499999999324, 5.692999999997991], 0.3: [6.919999999994555, -9.648999999998258, -14.166999999989637, -4.13100000000272], 0.35: [-12.05199999999988, -5.283000000003792, -15.354999999996217, -2.5960000000031145], 0.4: [-22.604999999976442, -11.820999999997827, -13.502999999996526, -7.626000000004815], 0.45: [-23.084999999985776, -18.585999999991653, -21.990999999988393, -14.480999999999213], 0.5: [-30.448999999971566, -18.719999999990783, -20.816999999976098, -10.14399999999253]}, 'action_only': {0.0: [95.71299999999309, 98.10599999999658, 95.26199999999324, 37.71800000001806], 0.05: [45.59700000000473, 14.79900000000871, 12.67200000000146, 97.08199999999457], 0.1: [27.26200000001424, -25.057999999976655, 22.236000000018684, 73.6269999999822], 0.15: [21.205000000012134, -28.10299999997768, 13.005000000003038, 21.320000000013874], 0.2: [3.9089999999933966, -14.19899999999675, -4.367000000002723, 17.24800000000297], 0.25: [-2.372000000001199, -13.050999999988969, -12.087000000003455, 4.192999999997368], 0.3: [-11.418999999998427, -1.6640000000012913, -10.876999999997102, -8.735000000001797], 0.35: [-20.84499999997825, -4.439000000002938, -21.595999999981736, -6.50700000000316], 0.4: [-26.629999999971503, -12.556999999992918, -16.447999999982166, -13.035000000000032], 0.45: [-33.95299999996912, -15.550999999994817, -18.254999999984303, 2.501000000001058], 0.5: [-31.98399999997388, -9.868000000005646, -20.011999999992884, -8.174000000008329]}, 'goal_only': {0.0: [96.15199999999308, 98.28499999999694, 95.30699999999737, 36.90200000001727], 0.05: [92.76999999999128, 98.30799999999635, 87.66099999999511, 96.15299999999375], 0.1: [95.00899999999201, 98.29899999999687, 71.66499999998472, 96.80699999999464], 0.15: [95.83599999999142, 98.40799999999666, 76.05899999998461, 97.48299999999506], 0.2: [97.01299999999526, 90.94199999999844, 77.02699999998795, 97.29299999999445], 0.25: [90.95999999999161, 82.07299999999212, 73.38599999998624, 93.3709999999952], 0.3: [92.56399999999427, 68.5939999999895, 70.50799999998975, 91.86599999999646], 0.35: [91.60499999999514, 68.64999999998932, 73.93199999997641, 82.83299999999204], 0.4: [89.95599999998824, 65.37299999999139, 74.64999999999227, 91.59099999998878], 0.45: [84.03199999998795, 46.100000000006176, 55.76899999999276, 79.86699999998373], 0.5: [84.17399999999448, 46.14699999999815, 56.5309999999838, 81.0779999999873]}, 'both_same': {0.0: [95.56899999999173, 96.68099999999669, 93.77699999999142, 47.458000000001945], 0.05: [45.79700000000359, 3.0119999999953735, 12.844000000012903, 94.18099999999133], 0.1: [20.786000000004275, -17.799999999996455, 33.101000000016626, 55.18999999999451], 0.15: [22.779000000010566, -18.380999999989058, 3.902999999997156, 35.871000000015854], 0.2: [14.389000000014635, -1.207000000003226, 1.997000000000214, 7.924999999994616], 0.25: [1.5169999999945125, -4.251000000001948, -5.3970000000040965, 4.044999999992927], 0.3: [-7.9980000000033655, -12.355999999984649, -2.2190000000001233, 6.360999999997257], 0.35: [-14.173999999985387, -16.470999999987818, -14.757999999997743, -5.02000000000602], 0.4: [-18.632999999987398, -22.025999999986553, -24.86299999997766, -11.641000000001457], 0.45: [-21.560999999987487, -14.444999999992708, -15.002999999988168, -6.317000000004302], 0.5: [-29.229999999970957, -21.994999999988707, -18.364999999979247, -3.786000000002772]}}
0, 1, 4, 5, 6

In [16]:
def eval_scale(agent, episode_durations):
    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.meta_controller.is_training = False
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for scale in np.arange(1.0,7.01,0.5):
        env = NormalizedEnv(PointPushEnv(scale))

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal_raw = agent.select_goal(g_state, False, False)
                goal = agent.convert_goal(goal_raw)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                    
                    next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    next_goal = agent.h(g_state, goal, g_next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    g_state = g_next_state
                    goal = next_goal

        episode_durations[np.round(scale, 2)].append(overall_reward / num_episodes)

In [17]:
episodes = {}
for scale in np.arange(1.0,7.01,0.5):
    episodes[np.round(scale, 2)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

for i in [0, 1, 4, 5, 6]:
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_scale(agent, episodes)
        print(f"{i} scale: {episodes}")

print("----")
print(f"scale: {episodes}")

0 scale: {1.0: [7.115999999998565], 1.5: [32.22500000001827], 2.0: [93.18999999999572], 2.5: [97.19199999999768], 3.0: [86.80099999998802], 3.5: [97.22399999999678], 4.0: [97.71099999999596], 4.5: [98.12999999999651], 5.0: [98.15499999999656], 5.5: [97.92099999999614], 6.0: [97.72099999999584], 6.5: [97.5699999999955], 7.0: [97.3289999999953]}
1 scale: {1.0: [7.115999999998565, -45.77199999998934], 1.5: [32.22500000001827, 1.0900000000010994], 2.0: [93.18999999999572, 64.20299999998775], 2.5: [97.19199999999768, 95.58999999999287], 3.0: [86.80099999998802, 97.40399999999545], 3.5: [97.22399999999678, 97.7669999999958], 4.0: [97.71099999999596, 97.66299999999578], 4.5: [98.12999999999651, 96.47199999999388], 5.0: [98.15499999999656, 96.19699999999321], 5.5: [97.92099999999614, 96.77899999999428], 6.0: [97.72099999999584, 96.3309999999934], 6.5: [97.5699999999955, 96.12799999999297], 7.0: [97.3289999999953, 95.7909999999924]}
4 scale: {1.0: [7.115999999998565, -45.77199999998934, -6.8560

In [14]:
def eval_starting_position(agent, episode_durations):
    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.meta_controller.is_training = False
    agent.controller.is_training = False

    num_episodes = 100

    c = 10

    for extra_range in np.arange(0.0, 0.401, 0.05):
        
        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            extra = np.random.uniform(-0.1 - extra_range, 0.1 + extra_range, env.starting_point.shape)
            #extra = np.random.uniform(0.1, 0.1 + extra_range, env.starting_point.shape)
            #extra = extra * (2*np.random.randint(0,2,size=env.starting_point.shape)-1)
            env.unwrapped.state = np.array(env.starting_point + extra, dtype=np.float32)
            env.unwrapped.state[2] += math.pi / 2. # start facing up
            env.unwrapped.state[2] = env.state[2] % (2 * math.pi)
            observation = env.normalised_state()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal_raw = agent.select_goal(g_state, False, False)
                goal = agent.convert_goal(goal_raw)

                goal_done = False
                while not done and not goal_done:
                    action = agent.select_action(state, goal, False, False)
                    observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                    
                    next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    next_goal = agent.h(g_state, goal, g_next_state)
                                      
                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    #goal_done = agent.goal_reached(action, goal)
                    goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                    if (episode_steps % c) == 0:
                        goal_done = True

                    state = next_state
                    g_state = g_next_state
                    goal = next_goal

        episode_durations[np.round(extra_range, 3)].append(overall_reward / num_episodes)

In [15]:
episodes = {}
for extra_range in np.arange(0.0, 0.401, 0.05):
    episodes[np.round(extra_range, 3)] = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

for i in [0, 1, 4, 5, 6]:
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_starting_position(agent, episodes)
        print(f"{i} range: {episodes}")

print("----")
print(f"range: {episodes}")

0 range: {0.0: [98.13799999999655], 0.05: [98.01999999999575], 0.1: [97.54299999999455], 0.15: [98.08199999999626], 0.2: [95.97099999999443], 0.25: [91.63899999999921], 0.3: [88.13699999999224], 0.35: [73.33599999999042], 0.4: [80.5439999999849]}
1 range: {0.0: [98.13799999999655, 97.73499999999599], 0.05: [98.01999999999575, 97.4419999999954], 0.1: [97.54299999999455, 97.52999999999537], 0.15: [98.08199999999626, 93.10999999999285], 0.2: [95.97099999999443, 93.16399999999008], 0.25: [91.63899999999921, 76.7709999999792], 0.3: [88.13699999999224, 81.31099999998831], 0.35: [73.33599999999042, 50.285999999997784], 0.4: [80.5439999999849, 42.949000000004396]}
4 range: {0.0: [98.13799999999655, 97.73499999999599, 96.12599999999301], 0.05: [98.01999999999575, 97.4419999999954, 83.46699999998113], 0.1: [97.54299999999455, 97.52999999999537, 59.323999999992296], 0.15: [98.08199999999626, 93.10999999999285, 57.4249999999942], 0.2: [95.97099999999443, 93.16399999999008, 27.87500000000848], 0.25

In [ ]:
state_max = torch.from_numpy(env.observation_space.high).to(device).float()
state_min = torch.from_numpy(env.observation_space.low).to(device).float()
state_mid = (state_max + state_min) / 2.
state_range = (state_max - state_min)
def save_trajectories(agent, episode_durations, dirty):
    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 500
    agent.meta_controller.is_training = False
    agent.controller.is_training = False

    num_episodes = 10

    c = 10

    l2norm = 0.3
    episode_durations.append([])
    
    for i_episode in range(num_episodes):
        path = {"overall_reward": 0, "manager": [], "worker": []}

        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        g_state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)

        if dirty:
            g_state = g_state + state_range * noise
            g_state = torch.max(torch.min(g_state, state_max), state_min).float()
        if dirty:
            state = state + state_range * torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
            state = torch.max(torch.min(state, state_max), state_min).float()

        episode_steps = 0
        overall_reward = 0
        done = False
        while not done:
            # select a goal
            goal = agent.select_goal(g_state, False, False)
            path["manager"].append((episode_steps, g_state_.detach().cpu().squeeze(0).numpy(), goal.detach().cpu().squeeze(0).numpy()))

            goal_done = False
            while not done and not goal_done:
                action = agent.select_action(state, goal, False, False)
                path["worker"].append((episode_steps, torch.cat([state_, goal], 1).detach().cpu().squeeze(0).numpy(), action.detach().cpu().squeeze(0).numpy()))
                observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
                
                next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                g_next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                g_state_ = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                noise = torch.FloatTensor(state.shape).uniform_(-l2norm, l2norm).to(device)
                if dirty:
                    g_next_state = g_next_state + state_range * noise
                    g_next_state = torch.max(torch.min(g_next_state, state_max), state_min).float()
                if dirty:
                    next_state = next_state + state_range * torch.FloatTensor(next_state.shape).uniform_(-l2norm, l2norm).to(device)
                    next_state = torch.max(torch.min(next_state, state_max), state_min).float()

                next_goal = agent.h(g_state, goal, g_next_state)
                                  
                overall_reward += reward

                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                episode_steps += 1

                #goal_done = agent.goal_reached(action, goal)
                goal_reached = agent.goal_reached(g_state, goal, g_next_state)

                if (episode_steps % c) == 0:
                    goal_done = True

                state = next_state
                g_state = g_next_state
                goal = next_goal

        path["overall_reward"] = overall_reward
        episode_durations[-1].append(path)

In [ ]:
episodes = []

n_observations = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]

i = 0
while i < 13:
    #agent = train_model()
    agent = HIRO(n_observations, n_actions).to(device)
    load_model(agent, f"hiro_push_{i}")

    if agent is not None:
        # goal_attack, action_attack, same_noise
        save_trajectories(agent, episodes, True)
        #print(f"{i} paths: {episodes}")
        i += 1

print("----")
#print(f"paths: {episodes}")

episodes.pop(1)
episodes.pop(5 - 1)
episodes.pop(11 - 2)

torch.save(episodes, "PointPush_dirty_eps.pt")

----
